In [ ]:
import os
import numpy as np
import tensorflow.keras
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import  Activation, Dropout,SpatialDropout1D, Conv3D, ConvLSTM2D, Conv1D, TimeDistributed, Dense, Input, Conv2D, Flatten, MaxPooling2D, Reshape, Conv2DTranspose, BatchNormalization
import tensorflow_addons as tfa
from tensorflow.keras.callbacks import ModelCheckpoint
from PIL import Image, ImageSequence
import pylab as plt
import random
# from google.colab import drive
# drive.mount('/content/gdrive')
#PATH_OF_DATA= '/content/gdrive/"My Drive"/Autoencoders_model_Myrto/autoencoders'
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.metrics import mean_squared_error
from math import sqrt
from scipy.stats.stats import pearsonr
from sklearn.metrics import mean_absolute_error
import sys
from PIL import GifImagePlugin
from skimage.color import rgb2gray

In [ ]:
window_TCN=8
window_ConvLSTM=18

In [ ]:
# C O M P L E T E   D A T A S E T
frames=[]
for i in range(2003,2020):
  # img = Image.open('/content/gdrive/My Drive/monthly_day/d_%i.gif' %i)
  # img = Image.open('soil_moisture_new\sweden_28_5km_data\%i.gif' %i)
  # img = Image.open('sweden_60_1km\sweden_60_1km_data\%i.gif' %i)
  img = Image.open('sweden_28_5km\sweden_28_5km_data\%i.gif' %i)
  # img = Image.open('america\\america_140_1km\\america_140_1km_data\\%i.gif' %i)
  

  img.load()
    
  for frame in ImageSequence.Iterator(img):
      
    frame_2 = frame.convert('RGB')    #(28, 28, 3)
    frame_2 = np.array(frame_2)       #(28, 28, 3)
    frame_2 = np.sum(frame_2, axis=2) #(28, 28)
    frame_2 = frame_2.reshape(frame_2.shape[0], frame_2.shape[1], 1) #( 28, 28, 1)
    frames.append(frame_2) 
frames = np.array(frames, dtype='uint8')  #(210, 28, 28, 1)
print(frames.max(),frames.min())

d=np.mean(frames,axis=1)
e=np.mean(d,axis=2)
g=np.mean(d,axis=1)
plt.plot(g)
plt.show()

e=(g-173)/3
plt.plot(e[1:25])
plt.show()

sys.exit()

plt.plot((frames[:,14,14,0]-273)/10)

sys.exit()

In [ ]:
maxi=np.max(frames)
mini=np.min(frames)
frames = (frames-mini)/(maxi-mini) #(209, 28, 28, 1)

In [ ]:
# T R A I N   D A T A 
train_frames=frames[0:192,:,:,:] #(209, 28, 28, 1)

maxi=np.max(train_frames)
mini=np.min(train_frames)
train_frames = (train_frames-mini)/(maxi-mini) #(209, 28, 28, 1)

In [ ]:
X_train_ConvLSTM, y_train_ConvLSTM = [], []
for i in range(len(train_frames)-window_ConvLSTM):
  X_train_ConvLSTM.append(train_frames[i:i+window_ConvLSTM,::,::,::])
  y_train_ConvLSTM.append(train_frames[i+1:i+window_ConvLSTM+1,::,::,::])
X_train_ConvLSTM=np.array(X_train_ConvLSTM)  
y_train_ConvLSTM=np.array(y_train_ConvLSTM) 

In [ ]:
X_train_TCN, y_train_TCN = [], []
for i in range(len(train_frames)-window_TCN):
  X_train_TCN.append(train_frames[i:i+window_TCN,::,::,::])
  y_train_TCN.append(train_frames[i+1:i+window_TCN+1,::,::,::])
X_train_TCN=np.array(X_train_TCN)  
y_train_TCN=np.array(y_train_TCN) 

# V A L I D A T I O N   D A T A 
r=random.sample(range(len(train_frames)-window-1), 20)
X_val=X_train[r, ::, ::, ::, ::] 
y_val=y_train[r, ::, ::, ::, ::]  
X_train = np.delete(X_train, (r), axis=0) 
y_train = np.delete(y_train, (r), axis=0) 

print(np.shape(X_train))
print(np.shape(y_train))

In [ ]:
from tensorflow import keras

ae_ConvLSTM = keras.models.load_model('sweden_60_1km\\ConvLSTM\\vgg.h5')
ae_TCN = keras.models.load_model('sweden_60_1km\\TCN\\vgg.h5')

ae_ConvLSTM = keras.models.load_model('sweden_28_5km\\ConvLSTM\\vgg.h5')
ae_TCN = keras.models.load_model('sweden_28_5km\\TCN\\vgg.h5')

ae_ConvLSTM = keras.models.load_model('america\\america_140_1km\\ConvLSTM\\vgg_112_112.h5')
ae_TCN = keras.models.load_model('america\\america_140_1km\\TCN\\vgg_112_112.h5')

In [ ]:
ae_ConvLSTM = keras.models.load_model('ae_ConvLSTM.h5')
ae_TCN = keras.models.load_model('ae_TCN.h5')

ae_ConvLSTM.compile(optimizer='adam', loss="mse")
ae_TCN.compile(optimizer='adam', loss="mse")

history_ConvLSTM=ae_ConvLSTM.fit(X_train_ConvLSTM, y_train_ConvLSTM, epochs=100)
history_TCN=ae_TCN.fit(X_train_TCN, y_train_TCN, epochs=100)

ae_ConvLSTM.save('ae_ConvLSTM.h5')
ae_TCN.save('ae_TCN.h5')

history_ConvLSTM=ae_ConvLSTM.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val))
history_TCN=ae_TCN.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val))

In [ ]:
num_days_test=12

In [ ]:
[n1,n2,n3,n4]=frames.shape

In [ ]:
pearsonr_lst_ConvLSTM=np.zeros([num_days_test,1])
mean_absolute_error_lst_ConvLSTM=np.zeros([num_days_test,1])
mean_squared_error_lst_ConvLSTM=np.zeros([num_days_test,1])

In [ ]:
pearsonr_lst_TCN=np.zeros([num_days_test,1])
mean_absolute_error_lst_TCN=np.zeros([num_days_test,1])
mean_squared_error_lst_TCN=np.zeros([num_days_test,1])

In [ ]:
img_ConvLSTM=np.zeros([num_days_test,n2,n3])
img_gt=np.zeros([num_days_test,n2,n3])
img_TCN=np.zeros([num_days_test,n2,n3])

In [ ]:
for i in range(0,num_days_test):

    # T E S T   D A T A
    smps_ConvLSTM=np.arange(192-window_ConvLSTM+i,192+i)
    test_frames_ConvLSTM = frames[smps_ConvLSTM,:,:,:]
    # test_frames_ConvLSTM = (test_frames_ConvLSTM-mini)/(maxi-mini)
    test_frames_ConvLSTM = np.expand_dims(test_frames_ConvLSTM, axis=0)
    
    smps_TCN=np.arange(192-window_TCN+i,192+i)
    test_frames_TCN = frames[smps_TCN,:,:,:]
    # test_frames_TCN = (test_frames_TCN-mini)/(maxi-mini)
    test_frames_TCN = np.expand_dims(test_frames_TCN, axis=0)
    
    
    gt = frames[192+i,:,:,:]
    img_gt[i,:,:]=gt[:,:,0]
    # gt = (gt-mini)/(maxi-mini)
    
    # P R E D I C T I O N
    new_pos_ConvLSTM=ae_ConvLSTM.predict(test_frames_ConvLSTM) #(1, window, 28, 28, num_channels to predict)
    new_pos_TCN=ae_TCN.predict(test_frames_TCN) #(1, window, 28, 28, num_channels to predict)
    
    # P L O T S
    # fig = plt.figure(figsize=(10, 5))
    # ax = fig.add_subplot(121)
    # plt.imshow((new_pos[0,-1,:,:,0])) 
    # ax = fig.add_subplot(122)
    # plt.imshow(gt)
    # plt.show()

    # plt.savefig('prediction1.png' )
    
    test=gt.flatten()
    
    img_ConvLSTM[i,:,:]=new_pos_ConvLSTM[0,-1,::,::,0]
    img_TCN[i,:,:]=new_pos_TCN[0,-1,::,::,0]
    
    
    pred_ConvLSTM=new_pos_ConvLSTM[0,-1,::,::,0].flatten()
    pearsonr_lst_ConvLSTM[i]=pearsonr(test, pred_ConvLSTM)[0]
    mean_absolute_error_lst_ConvLSTM[i]=mean_absolute_error(test, pred_ConvLSTM)
    mean_squared_error_lst_ConvLSTM[i]=mean_squared_error(test, pred_ConvLSTM)
    
    pred_TCN=new_pos_TCN[0,-1,::,::,0].flatten()
    pearsonr_lst_TCN[i]=pearsonr(test, pred_TCN)[0]
    mean_absolute_error_lst_TCN[i]=mean_absolute_error(test, pred_TCN)
    mean_squared_error_lst_TCN[i]=mean_squared_error(test, pred_TCN)
    
    # print(pearsonr(test, pred_ConvLSTM))
    # print(mean_absolute_error(test, pred_ConvLSTM))
    # print(mean_squared_error(test, pred_ConvLSTM))

In [ ]:
labels = ['1', '2', '3', '4','5','6','7','8','9','10','11','12']
x=np.arange(0,num_days_test)
plt.plot(x,pearsonr_lst_ConvLSTM, label='ConvLSTM')
plt.plot(x,pearsonr_lst_TCN, label='Embedded TCN')
plt.xlabel("Month (2020)")
plt.ylabel("PCC")
plt.xticks(x, labels)
plt.legend()
plt.show()

In [ ]:
labels = ['1', '2', '3', '4','5','6','7','8','9','10','11','12']
x=np.arange(0,num_days_test)
plt.plot(x,mean_absolute_error_lst_ConvLSTM, label='ConvLSTM')
plt.plot(x,mean_absolute_error_lst_TCN, label='Embedded TCN')
plt.xlabel("Month (2020)")
plt.ylabel("MAE")
plt.xticks(x, labels)
plt.legend()
plt.show()

a=np.mean(new_pos_ConvLSTM,axis=2)
b=np.mean(a,axis=2)
c=b[0,:,0]
plt.plot(c)
plt.plot((c*(maxi-mini)-mini)/10)
plt.show()

In [ ]:
d=np.mean(img_ConvLSTM*maxi,axis=-1)
e=np.mean(d,axis=-1)
e=(e-173)/5
plt.plot(e,label='ConvLSTM')
# plt.show()

In [ ]:
d=np.mean(img_TCN*maxi,axis=-1)
e=np.mean(d,axis=-1)
e=(e-173)/5
plt.plot(e,label='Embedded TCN')
# plt.show()

In [ ]:
d=np.mean(img_gt*maxi,axis=-1)
e=np.mean(d,axis=-1)
e=(e-173)/5
plt.plot(e,label='Ground Truth values')
plt.legend()
plt.xticks(x, labels)
plt.xlabel("Month (2020)")
plt.ylabel("LST at daytime ($^\circ$C) ")
plt.show()

In [ ]:
from scipy import stats
# stats.ttest_ind((img_ConvLSTM).flatten(),(img_TCN).flatten())
print(stats.ttest_rel((img_ConvLSTM-img_gt).flatten(),(img_TCN-img_gt).flatten()))
# print(stats.ttest_rel((img_ConvLSTM[:,:,:]).flatten(),(img_TCN[:,:,:]).flatten()))

plt.plot((((test_frames_ConvLSTM[0,:,14,14,0])*(maxi-mini))-mini)/10)
plt.plot(mean_absolute_error_lst)
plt.xlabel("Month")
plt.ylabel("MAE")
plt.show()
plt.plot(mean_squared_error_lst)
plt.show()